# Assignment 3

In [1]:
import pandas as pd
import numpy as np


# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2        0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 3        4.0  0.0  5.0  0.0  1.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0  0.0   
 4        3.0  0.0  4.0  0.0  0.0  3.0  2.0  2.0  0.0  5.0  ...  0.0  4.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      0.0  4.0  0.0  5.0  0.0  4.0  0.0  4.0  0.0  4.0  ...  0.0  0.0  0.0   
 296      0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  3.0  5.0  ...  0.0  0.0  0.0   
 297      4.0  0.0  4.0  0.0  0.0  4.0  3.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 298      0.0  0.0  0.0  3.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  5.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solutioin in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9

# This function helps in calculating the pearson correlation coefficient for both item and user based method
def correlation_computation(data,factor):
    corr = np.zeros((data.shape[0],data.shape[0]))

    # Nested for loops to calculate the similarity of each user with every other user in imputed training set
    for i,vec_a in enumerate(data):
        for j,vec_b in enumerate(data):

            # To avaoid calculating similarity of user with itself
            if (i == j):
                continue

            # ratings corated by the current pair of users
            mask_a = vec_a > 0
            mask_b = vec_b > 0

            # corrated item index, skip if there are no corrated ratings
            corrated_index = np.intersect1d(np.where(mask_a), np.where(mask_b))
            if len(corrated_index) == 0:
                continue

            # average value of user_a_vec and user_b_vec
            mean_a = np.sum(vec_a) / (np.sum(np.clip(vec_a, 0, 1)) + EPSILON)
            mean_b = np.sum(vec_b) / (np.sum(np.clip(vec_b, 0, 1)) + EPSILON)

            # compute pearson corr
            a_subtract_mean = vec_a[corrated_index] - mean_a
            b_subtract_mean = vec_b[corrated_index] - mean_b

            a_sub_mean_square = np.square(a_subtract_mean)
            b_sub_mean_square = np.square(b_subtract_mean)

            sqrt_a_summ = np.sqrt(np.sum(a_sub_mean_square))
            sqrt_b_summ = np.sqrt(np.sum(b_sub_mean_square))

            sim = np.sum(a_subtract_mean * b_subtract_mean) / ((sqrt_a_summ * sqrt_b_summ) + EPSILON)

            # significance weighting
            weighted_sim = (min(len(corrated_index), factor) / factor) * sim
            corr[i][j] = weighted_sim
            
    return corr

# STEP 1 USER Based pearson correlation computaion
imputed_user_pearson_corr = correlation_computation(imputed_train_ds, GAMMA)

# Step 2 ITEM based pearson correlation computaion
imputed_train_ds_transpose = imputed_train_ds.transpose()
imputed_item_pearson_corr = correlation_computation(imputed_train_ds_transpose, DELTA)

# Step 3
# Predict the values
for (user,item), rating in np.ndenumerate(imputed_train_ds):
    if(rating == 0):
        # Similar neighbor selection for users and items
        similar_users_index = np.where(imputed_user_pearson_corr[user] > ITA)
        similar_items_index = np.where(imputed_item_pearson_corr[item] > THETA)
        
        # Coefficient values of similar users and items
        similar_users_corr = imputed_user_pearson_corr[user][similar_users_index]
        similar_items_corr = imputed_item_pearson_corr[item][similar_items_index]
        
        # Average value of current users and current items ratings
        sim_users_ratings = imputed_train_ds[similar_users_index]
        sim_items_ratings = imputed_train_ds_transpose[similar_items_index]
        
        user_mean = np.sum(imputed_train_ds[user]) / (np.sum(np.clip(imputed_train_ds[user], 0, 1)) + EPSILON)
        item_mean = np.sum(imputed_train_ds_transpose[item]) / (np.sum(np.clip(imputed_train_ds_transpose[item], 0, 1)) + EPSILON)
        
        sim_user_mean = np.sum(sim_users_ratings, axis=1) / (np.sum(np.clip(sim_users_ratings, 0, 1), axis=1) + EPSILON)
        sim_item_mean = np.sum(sim_items_ratings, axis=1) / (np.sum(np.clip(sim_items_ratings, 0, 1), axis=1) + EPSILON)
        
        #Select users who rated item 
        mask_users_who_rated_item = sim_users_ratings[:,item] > 0

        #Select items rated by users
        mask_items_rated_by_user = sim_items_ratings[:,user] > 0

        # sim(u, v) * (r_v - mean_v)
        sim_users_r_sum_mean = similar_users_corr[mask_users_who_rated_item] * (sim_users_ratings[mask_users_who_rated_item,item] 
                                                                                - sim_user_mean[mask_users_who_rated_item])
        sim_items_r_sum_mean = similar_items_corr[mask_items_rated_by_user] * (sim_items_ratings[mask_items_rated_by_user,user]
                                                                               - sim_item_mean[mask_items_rated_by_user])
        
        # prediction component for users and item based method
        p_user = user_mean + ((np.sum(sim_users_r_sum_mean)) / (np.sum(similar_users_corr[mask_users_who_rated_item]) + EPSILON))
        p_item = item_mean + ((np.sum(sim_items_r_sum_mean)) / (np.sum(similar_items_corr[mask_items_rated_by_user]) + EPSILON))            
        
        # Now we use lambda to accomodate both user and item based prediction to give overall prediction
        if(len(similar_users_index[0])!=0 and len(similar_items_index[0])!=0):
            imputed_train_ds[user][item] = (LAMBDA*p_user)+((1-LAMBDA)*p_item)
        if(len(similar_users_index[0])!=0 and len(similar_items_index[0])==0):
            imputed_train_ds[user][item] = p_user
        if(len(similar_users_index[0])==0 and len(similar_items_index[0])!=0):
            imputed_train_ds[user][item] = p_item
        if(len(similar_users_index[0])==0 and len(similar_items_index[0])==0):
            imputed_train_ds[user][item] = 0

        # CLiping the values if the prediction is less than 0 or more than 5
        imputed_train_ds[user][item] = np.clip(imputed_train_ds[user][item], 0, 5)

# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,4.098039,4.599123,4.000000,4.000000,4.000000,3.279570,3.748317,4.000000,3.192308,0.000000,...,3.468862,3.795833,3.981798,3.944617,2.699812,3.301871,0.000000,3.679737,3.225485,3.599483
1,4.000000,3.903199,4.052185,2.000000,4.042097,3.758549,4.256014,4.364840,3.736341,3.752469,...,3.767700,3.906263,3.831515,2.457485,3.335538,3.578217,3.846081,3.325431,3.597180,3.727359
2,4.013856,4.053690,4.063585,3.930933,5.000000,3.770952,4.232758,3.790633,3.745245,4.000000,...,3.778638,3.830291,3.532698,3.827631,3.508600,3.618484,3.855070,3.872170,3.681076,3.847909
3,4.000000,3.416356,5.000000,3.817611,1.000000,3.671805,3.000000,2.000000,3.212253,0.000000,...,2.831714,3.103488,3.398496,4.000000,1.000000,1.657626,0.000000,4.042790,2.946554,2.000000
4,3.000000,2.164870,4.000000,3.817611,1.232226,3.000000,2.000000,2.000000,3.732539,5.000000,...,2.750076,4.000000,3.424521,3.235659,2.875999,2.369788,0.000000,3.772917,2.695387,3.153400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.088522,4.000000,3.879772,5.000000,3.446614,4.000000,3.694052,4.000000,3.765721,4.000000,...,2.794995,3.204497,3.907079,3.606444,3.042521,3.209744,3.260831,3.922136,3.364028,3.639053
296,4.824394,2.292862,4.150394,3.862642,3.608517,3.371180,3.922789,5.000000,3.000000,5.000000,...,3.439765,3.573233,3.826591,4.117533,1.277887,4.469653,0.000000,3.822679,3.843203,3.647896
297,4.000000,3.205203,4.000000,3.862642,3.608517,4.000000,3.000000,4.000000,2.822167,0.000000,...,3.126192,3.607093,3.866614,3.038308,3.097433,3.116938,0.000000,4.080181,3.077990,3.174425
298,4.125615,3.701817,4.150394,3.000000,3.608517,4.535220,3.922789,4.000000,3.408609,0.000000,...,3.165336,3.107413,5.000000,3.398896,3.097433,3.000000,0.000000,3.857930,3.171472,3.161344


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair of users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0
                    
        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue
        
        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)
        
        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j
        
        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)
        
        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))
        
        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)
        
        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim
        
        active_user_pearson_corr[i][j] = weighted_sim
        
active_user_pearson_corr

array([[-0.1125491 , -0.06956752,  0.1084332 , ..., -0.2788627 ,
         0.03648109, -0.24983521],
       [ 0.37830306,  0.16926814,  0.0150141 , ...,  0.29501595,
         0.33327391,  0.22398209],
       [ 0.55440228, -0.04634003,  0.06170993, ..., -0.16265582,
        -0.30250785,  0.5003653 ],
       ...,
       [ 0.51979302,  0.30628048,  0.48869684, ..., -0.53986397,
         0.57442337, -0.16922682],
       [ 0.36057392, -0.26396115,  0.3698099 , ..., -0.19516492,
         0.04006666,  0.18364036],
       [ 0.12854737,  0.13329892, -0.4069471 , ..., -0.13625667,
         0.08369348, -0.28824035]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred

array([[0.        , 0.        , 0.        , ..., 0.        , 3.10074496,
        0.        ],
       [0.        , 3.87383477, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.70971555, 0.        , 3.79470159, ..., 3.43542284, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.61144965, 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7631975894994888, RMSE: 0.9807800935517199
